In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import pandas as pd
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [2]:
#import dataset from data.csv file
df = pd.read_csv('Dummy_Month.csv')
df.head()

,Unnamed: 0,Bay_id,Status,Hour,Minutes,DayOfWeek,Date
0,0,1001,Unoccupied,0,0,Wednesday,2021-08-04
1,1,1002,Present,0,0,Wednesday,2021-08-04
2,2,1003,Present,0,0,Wednesday,2021-08-04
3,3,1004,Unoccupied,0,0,Wednesday,2021-08-04
4,4,1005,Present,0,0,Wednesday,2021-08-04


In [3]:
#The amount of observations (rows) and characteristics(columns) in a dataset
print ('The Total Rows in the dataset = ' + str(df.shape[0]))
print ('The Total Columns in the dataset = ' + str(df.shape[1]))

The Total Rows in the dataset = 13440
The Total Columns in the dataset = 7


In [3]:
Bay_ID = (df['Bay_id'] == 1001)
Multiple_Bay_ID = (df['Bay_id'].isin([1001, 1003, 1005]))
Parking_Status = (df['Status'] == 'Present')
Day_Of_Week = (df['DayOfWeek'] == 'Wednesday')
Date = (df['Date'] == '2021-08-04')
Hour = (df['Hour'] == 10)

## 7-Day Expected Parking Availability

In [4]:
CountStatus = df.loc[Multiple_Bay_ID & Parking_Status].groupby('Status').DayOfWeek.value_counts().unstack(0)

CountStatus = CountStatus.reset_index()
CountStatus['Present'] = (CountStatus['Present'] / 4).round(0)
CountStatus.columns = ['Day', 'Mean_Parking_Availabilities']
CountStatus = CountStatus.set_index('Day').reindex(['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']).reset_index()

CountStatus

,Day,Mean_Parking_Availabilities
0,Monday,144.0
1,Tuesday,145.0
2,Wednesday,145.0
3,Thursday,149.0
4,Friday,142.0
5,Saturday,138.0
6,Sunday,137.0


In [5]:
Currentday_Count = df.loc[Multiple_Bay_ID & Parking_Status & Date].groupby('Status').DayOfWeek.value_counts().unstack(0)
Currentday_Count = Currentday_Count.reset_index()
Currentday_Count.columns = ['Day', 'Parking_Availabilities']

Day_of_Choice = Currentday_Count.iloc[0,0]

Currentday_Count

,Day,Parking_Availabilities
0,Wednesday,139


In [6]:
CountStatus_categories = CountStatus['Day'].tolist()
CountStatus_Values = CountStatus['Mean_Parking_Availabilities'].tolist()

Currentday_Count_categories = Currentday_Count['Day'].tolist()
Currentday_Count_Values = Currentday_Count['Parking_Availabilities'].tolist()

In [41]:
fig = go.Figure()

fig.add_trace(go.Scatterpolar(
      r=CountStatus_Values,
      theta=CountStatus_categories,
      fill='toself',
      name='Expected '))
fig.add_trace(go.Scatterpolar(
      r=Currentday_Count_Values,
      theta=Currentday_Count_categories,
      fill='toself',
      name='Current'))

fig.update_layout(polar=dict(radialaxis=dict(visible=True,range=[0, 160])), showlegend=True, 
                  title=go.layout.Title(text='7-Day Expected Parking Availability'))

fig.show()